In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [ ]:
# Activation Functions
def tanh(z):
    return np.tanh(z)

In [ ]:
def tanh_d(z):
    return 1 - (np.tanh(z))**2

In [ ]:
def relu(z):
    return np.maximum(0, z)

In [ ]:
def relu_d(z):
    return z > 0

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
def sigmoid_d(z):
    s = sigmoid(z)
    d = s * (1 - s)
    return d

In [ ]:
def softmax(z):
    z_ = np.exp(z)
    y_pred = z_ / z_.sum(axis=1, keepdims=True)
    return y_pred

In [ ]:
# Helper function to predict labels
def predict(y_pred):
    return np.argmax(y_pred, axis=1)

In [ ]:
# Helper function to calculate hits and accuracy
def hits(y_pred, y):
    count = 0
    s = set()
    y = predict(y)
    y_pred = predict(y_pred)
    for i in range(y.shape[0]):
        if(y[i] == y_pred[i]):
            s.add(y[i])
            count += 1
    accuracy = (count / m) * 100
    print(s)
    return count, accuracy

In [ ]:
# Gradient Descent for updating weights and biases
def gradient_descent(a_in, y, a_out, w, b, lr):
    err = a_out - y
    dw = err.T @ a_in / m
    db = np.sum(err, axis=0) / m
    w = w - lr * dw
    b = b - lr * db
    return w, b, err

In [ ]:
# Forward Propagation
def forward_prop(a_in, w, b, ltype):
    a_out = a_in @ w.T + b
    if ltype == "tanh":
        return tanh(a_out)
    elif ltype == "relu":
        return relu(a_out)
    elif ltype == "softmax":
        return softmax(a_out)
    elif ltype == "sigmoid":
        return sigmoid(a_out)
    else:
        return a_out

In [ ]:
# Backward Propagation
def backward_prop(a_in, a_out, prev_err, w, ltype):
    err = prev_err @ w

    if ltype == "relu":
        err *= relu_d(a_out)
    elif ltype == "tanh":
        err *= tanh_d(a_out)
    elif ltype == "sigmoid":
        err *= sigmoid_d(a_out)

    dw = err.T @ a_in / m
    db = np.sum(err, axis=0) / m
    return dw, db, err

In [ ]:
# Neural Network Training
def neural_network(x, y, n, units, ltype, lr, itera):
    w = []
    b = []
    activation = []
    activation.append(x)

    for j in range(itera+1):
        activation[0] = x

        for i in range(n):
            if j == 0:
                wi = np.random.randn(units[i], activation[i].shape[1])
                bi = np.zeros((1, units[i]))
                w.append(wi)
                b.append(bi)
                ai = forward_prop(activation[i], w[i], b[i], ltype[i])
                activation.append(ai)
            else:
                activation[i+1] = forward_prop(
                    activation[i], w[i], b[i], ltype[i])

        for i in range(n, 0, -1):
            if i == n:
                w[n-1], b[n-1], err1 = gradient_descent(
                    activation[n-1], y, activation[n], w[n-1], b[n-1], lr)
            else:
                dw, db, err1 = backward_prop(
                    activation[i-1], activation[i], err1, w[i], ltype[i-1])
                w[i-1] = w[i-1] - dw * lr
                b[i-1] = b[i-1] - db * lr

        if j % 10 == 0:
            count, accuracy = hits(activation[n], y)
            print(f"no. of correct predictions after {j} iterations: {count} and accuracy is: {accuracy}")
            print()

    return w, b

In [ ]:
# Load dataset
data = pd.read_csv("D:\\HP\\users\\OneDrive\\Desktop\\AI ML\\train and test\\classification_train.csv")
x = data.iloc[:29000, 2:].values / 255
y = data.iloc[:29000, 1:2].values
m, n = x.shape
lr = 1e-3
itera = 100
y2 = np.zeros((m, 10))
for i in range(m):
    y2[i, y[i]] = 1

In [ ]:
# User Input for Neural Network Architecture
n = int(input("Enter the number of layers in the neural network: "))
units = []
ltype = []
for i in range(n):
    unit = int(input(f"Enter the number of units in layer {i+1}: "))
    units.append(unit)
    ltyp = input(f"Enter the type of activation in layer {i+1}: ")
    ltype.append(ltyp)
    print("\n\n")

In [ ]:
# Train the Neural Network
w, b = neural_network(x, y2, n, units, ltype, lr, itera)

In [ ]:
# Data Visualization
fig, axes = plt.subplots(1, 1, figsize=(16, 5))

In [ ]:
# Visualize distribution of labels
axes.hist(predict(activation[n]), bins=np.arange(11)-0.5,rwidth=0.8, color='skyblue', edgecolor='black')
axes.set_title('Distribution of Predicted Labels')
axes.set_xlabel('Label')
axes.set_ylabel('Frequency')
plt.show()

In [ ]:
# Data Visualization
fig = plt.figure(figsize=(12, 8))
gs = gridspec.GridSpec(4, 5)
m, n = x.shape
indices = np.random.choice(m, size=20, replace=False)

In [ ]:
for i, index in enumerate(indices):
    ax = plt.subplot(gs[i])
    ax.imshow(x[index].reshape(28, 28), cmap='gray')  # Reshape for display
    ax.set_title(f"Predicted: {predict(activation[n])[index]}")
    ax.axis('off')

In [ ]:
plt.show()

In [ ]:
# Make predictions on the test set
x_test = data.iloc[29001:, 2:].values / 255
y_test = data.iloc[29001:, 1].values
y_pred_test = forward_prop(x_test, w[0], b[0], ltype[0])

In [ ]:
# Visualize distribution of predicted labels on the test set
fig, axes = plt.subplots(1, 1, figsize=(16, 5))
axes.hist(predict(y_pred_test), bins=np.arange(11)-0.5, rwidth=0.8, color='skyblue', edgecolor='black')
axes.set_title('Distribution of Predicted Labels on Test Set')
axes.set_xlabel('Label')
axes.set_ylabel('Frequency')
plt.show()

In [ ]:
# Data Visualization for the test set
fig = plt.figure(figsize=(12, 8))
gs = gridspec.GridSpec(4, 5)
m_test, _ = x_test.shape
indices_test = np.random.choice(m_test, size=20, replace=False)

In [ ]:
for i, index in enumerate(indices_test):
    ax = plt.subplot(gs[i])
    ax.imshow(x_test[index].reshape(28, 28), cmap='gray')  # Reshape for display
    ax.set_title(f"Predicted: {predict(y_pred_test)[index]} | Actual: {y_test[index]}")
    ax.axis('off')

In [ ]:
plt.show()

In [ ]:
# Calculate accuracy on the test set
count_test, accuracy_test = hits(y_pred_test, y_test)
print(f"Correct predictions on the test set: {count_test} and accuracy is: {accuracy_test}")